# Acceleration controller

Consider the system of a point body (vehicle) of mass $1$, moving without friction, and controlled by an external force.

The equations of the dynamics are given by

\begin{eqnarray*}
\left(\begin{array}{l}\ddot{x}\\ \ddot{y}\end{array}\right) & = & \gamma(t)
\end{eqnarray*}

$\gamma$ is assumed piecewise constant, with changing points at regular intervals (the control rate). At each changing points, the value of $\gamma$ is acquired from a controller. At regular intervals, the controller has access to the current simulation time $t$ and state of the system $(x,y,\dot{x},\dot{y})$. The controller can thus keep a history of its observations, and base its decisions on it (control loop).

In [1]:
%pylab widget

import logging
from functools import partial
from myutil.simpy import SimpySimulation
from ipyshow.odesimu import System
from ipyshow.odesimu.util import PIDController, buffered, blurred, period_hook

Populating the interactive namespace from numpy and matplotlib


## Vehicle class

In [2]:
#----------------------------------------------------------------------------------------------------
class Vehicle(System):
#----------------------------------------------------------------------------------------------------

  def __init__(self,control):
    r"""
Args:
  control: a function of time
    """
    self.control = control
    def fun(t,state):
      x,y,xʹ,yʹ = state
      xʺ,yʺ = control(t)
      return array((xʹ,yʹ,xʺ,yʺ))
    self.fun = fun
    self.jac = lambda t,state,c=array(((0.,0.,1.,0.),(0.,0.,0.,1.),(0.,0.,0.,0.),(0.,0.,0.,0.))): c

  def displayer(self,env,ax,bounds=None,**ka):
    vmin,vmax = bounds
    ax.set(xlim=(vmin[0],vmax[0]),ylim=(vmin[1],vmax[1]))
    ax.set_title(r'Vehicle under acceleration control',fontsize='x-small')
    ax.scatter((0,),(0,),c='k',marker='o')
    a_vehicle= ax.scatter((),(),c='b',marker='o',label='vehicle')
    a_tail, = ax.plot((),(),'y')
    ax.legend(fontsize='x-small')
    def disp():
      a_vehicle.set_offsets((env.state[:2],))
      a_tail.set_data(*env.cached_states[:2])
    return disp

  @staticmethod
  def makestate(x,y,xʹ=0.,yʹ=0.):
    r"""
Args:
  x,y (float): position of vehicle [m]
  xʹ,yʹ (float): speed of vehicle [m.sec^-1]
    """
    return array((x,y,xʹ,yʹ))

  launch_defaults = dict(period=.1,cache=(200,.1),max_step=.01,method='LSODA')

In [3]:
logging.basicConfig(level=logging.WARNING)

def target_displayer(env,ax):
  return lambda a=ax.scatter((),(),c='r',s=20,label='target'): a.set_offsets((target(env.now),))

target_period = 25
def target_still(t,spot=array((7.,5.))): return spot
def target_circle(t,R=4.,ω=2*pi/target_period):
  θ = ω*t
  return R*hstack((sin(θ),cos(θ)))
def target_cycloid(t,R=4.,r=2.,l=-5.,ω=2*pi/target_period):
  θ = ω*t
  return R*hstack((sin(θ),cos(θ)))+r*hstack((sin(l*θ),cos(l*θ)))

target = target_still
#target = buffered(T=target_period,N=int(target_period*100))(target_circle)
#target = buffered(T=target_period,N=int(target_period*100))(partial(target_cycloid,l=5.))

def launch(control_kw,blurred_kw=None,**ka):
  otarget = target if blurred_kw is None else blurred(**blured_kw)(target)
  control = PIDController(N=2,observe=(lambda t,state: otarget(t)-state[:2]),**control_kw)
  syst = Vehicle(control)
  return syst.launch(target_displayer,dict(bounds=((-8.,-8.),(8.,8.))),hooks=(partial(period_hook,control),),init_y=ini,**ka)

ini = dict(x=0.,y=0.,xʹ=0.,yʹ=0.)

SimpySimulation(
  launch(
    dict(gP=10.,gI=None,gD=30.),
    None, # dict(level=.05,offset=(7.,5.))
    jac=None,
  ),
  frame_per_stu=25,
  play_kw=dict(track=60.),
)